## Installation and Import of Libraries

In [ ]:
!pip install catboost

In [ ]:
!pip install --upgrade linear-tree


In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import random
random.state= 42
np.random.seed=42
random.seed=42
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Polygon, Point
import geopandas as gpd
import xgboost as xgb
from tqdm import tqdm
import gc
from collections import defaultdict
from itertools import combinations
from sklearn.feature_selection import RFE
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import warnings
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from lineartree import LinearTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import precision_score, recall_score
pd.set_option('display.max_columns', None)


## Connecting to Google Drive and Current Working Directory

In [ ]:
# connecting to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# connecting to current working directory
os.chdir('/content/drive/MyDrive')
print(f"current working directory: {os.getcwd()}")

## Loading Dataset (Train, Test, and Submission File)

In [ ]:
# loading datasets
train = pd.read_csv('train_erosion.csv')

In [ ]:
train.head(2)

## Exploratory Data Analysis

###  Class Distribution of Target Variable

In [ ]:

map_class = {0:'No Gully/badland', 1:'Gully', 2:'Badland', 3:'Landslides'}
train['classes'] = train['Code'].map(map_class)
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))
sns.countplot(x='classes', data=train, palette="husl")
plt.xlabel("Categories")
plt.ylabel("Frequency")
plt.title("Frequency Distribution of Column")
plt.tight_layout()
plt.show()

In [ ]:
# total size of the train dataset
train.shape

In [ ]:
# Defining Columns we won't be using
not_needed_columns = ['UUID', 'sample_id', 'Unnamed: 0']
bands_list = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal']


# Feature Engineering

#### Aggregation (mean) of all similar columns.

* So here I am taking all Blue, green, red, nir, swir1 and swir2 columns and multiplying them by their percentile and calculating their mean, then removing the original three columns themselves.

In [ ]:
columns_used_to_calculate_mean = []

for count, band in enumerate(bands_list):
    all_band_columns = [col for col in train.columns if bands_list[count] in col.lower()]

    # get the probabiltity distribution value for each of the bands
    value_1 = int(all_band_columns[0].split('_')[2][1:])/100
    value_2 = int(all_band_columns[1].split('_')[2][1:])/100
    value_3 = int(all_band_columns[2].split('_')[2][1:])/100

    # calculate mean
    train['mean_' + bands_list[count]] = ((train[all_band_columns[0]])*value_1 + (train[all_band_columns[1]])*value_2 + (train[all_band_columns[2]])*value_3)/3
    train['mean_' + bands_list[count]] = train['mean_' + bands_list[count]].round(2)


    # append the names of the columns used to calculate the mean
    for column in all_band_columns:
        columns_used_to_calculate_mean.append(column)

### Applying Aggregation to similar columns

In [ ]:
prefix_names = ['landform.moderate.mountain.smooth','wetlands.regularly-flooded','landform.upper.large.slope','sample','ndvi','nightlights.average',
 'landform.terrace.smooth.plateau','bs','landform.middle.large.slope','nos','landform.alluvial.plain.pediplain',
 'landform','landform.alluvial.or.coasttal.plain.pediplain','landform.steep.mountain.smooth','bioclim.var','slope','fapar','lcv.forest','cdr',
 'landform.moderate.mountain.rough','landform.hills.rough.in.small.and.large.scale.','wv','downslope.curvature','evi',
 'wetlands.permanent','clm','log1p.upstream.area','vbf','upslope.curvature','landform.alluvial.or.coasttal.plain.gentlest.lake.plain.playa',
 'lcv','landform.dissected.terrace.moderate.plateau','pos.openess','landform.slope.in.and.around.terrace.or.plateau','dtm','snow.prob','ndsi',
 'wetlands.cw','landform.hills.rough.in.small.and.large.scale','wetlands.groundwater-driven','landform.alluvial.fan..pediment..bajada..pediplain',
 'landform.steep.mountain.rough']

In [ ]:

agg_full_train = pd.DataFrame()

for col in tqdm(prefix_names):
    cols = [c for c in train.columns if col in c]

    train_df_agg = train[cols].agg(['mean', 'max', 'min', 'std', 'nunique', 'skew', 'kurt'], axis=1)


    train_df_agg.columns = [f'{col}_{c}' for c in train_df_agg.columns]

    agg_full_train = pd.concat([agg_full_train, train_df_agg], axis=1)


# append the aggregated data to the full_train dataset
train = pd.concat([train, agg_full_train], axis=1)

train.dropna(axis=1, inplace=True)


### Trying to see if aggregation (mean) of all similar columns would work.

* So here I am taking all Blue, green, red, nir, swir1 and swir2 columns and dividing by the number of columns and then I would remove the original three columns themselves.

In [ ]:
columns_used_to_calculate_mean = []

for count, band in enumerate(bands_list):
    all_band_columns = [col for col in train.columns if bands_list[count] in col.lower()]

    # get the probabiltity distribution value for each of the bands
    value_1 = int(all_band_columns[0].split('_')[2][1:])/100
    value_2 = int(all_band_columns[1].split('_')[2][1:])/100
    value_3 = int(all_band_columns[2].split('_')[2][1:])/100

    # calculate mean
    train['mean_' + bands_list[count]] = ((train[all_band_columns[0]])*value_1 + (train[all_band_columns[1]])*value_2 + (train[all_band_columns[2]])*value_3)/3
    train['mean_' + bands_list[count]] = train['mean_' + bands_list[count]].round(2)

    # append the names of the columns used to calculate the mean
    for column in all_band_columns:
        columns_used_to_calculate_mean.append(column)

## Topographic Factors

### Topographic Wetness Index (TWI)

#### Function for calculating TWI

In [ ]:
### Calculating slope
def calculate_slope(dem, cell_size):
    dz_dx = np.gradient(dem) / cell_size
    dz_dy = np.gradient(dem) / cell_size

    slope_rad = np.arctan(np.sqrt(dz_dx**2 + dz_dy**2))
    slope_deg = np.degrees(slope_rad)
    return slope_deg


def calculate_twi_from_csv(dem_df, cell_size):
    # Read DEM data from CSV
    dem = dem_df.values

    # Calculate slope
    slope_deg = calculate_slope(dem, cell_size)

    # Calculate TWI
    a = np.ones_like(dem) * cell_size**2
    twi = np.log(a / np.tan(np.radians(slope_deg)))
    return twi

#### Appyling TWI function to dataset

In [ ]:
# retrieving the elevation column
dtm_col = [col for col in train.columns if 'dtm_elev' in col.lower()]

# calculating TWI for train dataset
twi =  calculate_twi_from_csv(dem_df= train[dtm_col[0]], cell_size=30)
train['twi'] = twi

### Combined slope length and slope angle (LS-factor)

#### LS-Factor Function

In [ ]:
def calculate_slope_length_factor(slope_length, slope_steepness, m=0.4):
    ls_factor = (slope_length / 22.1)**m * (np.sin(np.radians(slope_steepness)) / 0.09)**1.3
    return ls_factor

def calculate_ls_factor_from_csv(df, cell_size):
    # Read DEM data from CSV
    slope_deg = df['slope_merit.dem_m_250m_s0..0cm_2017_2017_go_epsg.4326_v20231002'].values

    # Calculate slope length (assuming constant cell size)
    slope_length = np.sqrt(cell_size**2 + cell_size**2)

    # Calculate LS factor
    ls_factor = calculate_slope_length_factor(slope_length, slope_deg)
    return ls_factor


#### Applying LS-Factor Function to dataset

In [ ]:
train_ls = calculate_ls_factor_from_csv(train, 250)
train['ls_factor'] = train_ls

### SPI (Stream Power Index)

#### SPI Function

In [ ]:
def calculate_spi_from_csv(df):
    # Calculate slope
    slope_deg = df['slope_merit.dem_m_250m_s0..0cm_2017_2017_go_epsg.4326_v20231002'].values

    # Calculate SPI
    spi = np.sin(np.radians(slope_deg))
    return spi

#### Applying SPI function to dataset

In [ ]:
train_spi = calculate_spi_from_csv(train)
train['spi'] = train_spi

## Vegetation Indices

### SAVI (Soil Adjusted Vegetation Index)

In [ ]:
# SAVI = ((NIR - Red) / (NIR + Red + L)) x (1 + L)

train['mean_savi'] = ((train['mean_nir'] - train['mean_red']) / (train['mean_nir'] + train['mean_red'] + 0.5) * (1 + 0.5))

### WDVI (Weighted Difference Vegetation Index )
* WDVI = NIR - (slope x RED)

In [ ]:
# retrieve the slope col
slope_col = [col for col in train.columns if 'slope_merit' in col.lower()]

# calculate WDVI
train['WDVI'] = train['mean_nir'] - (train[slope_col[0]] * train['mean_red'])

### NDWI (Normalized Difference Water Index)

In [ ]:
# 𝑁𝐼𝑅−𝑆𝑊𝐼𝑅1𝑁𝐼𝑅+𝑆𝑊𝐼𝑅1

train['mean_ndwi'] = ((train['mean_nir'] - train['mean_swir1']) / (train['mean_nir'] + train['mean_swir1']))

### NDII (Normalized Difference Infrared Index )

In [ ]:
## 𝑆𝑊𝐼𝑅1−𝑆𝑊𝐼𝑅2𝑆𝑊𝐼𝑅1+𝑆𝑊𝐼𝑅2

train['mean_ndii'] = ((train['mean_swir1'] - train['mean_swir2']) / (train['mean_swir1'] + train['mean_swir2']))

### SIWSI (Shortwave Infrared Water Stress Index)

In [ ]:
# 𝑁𝐼𝑅−𝑆𝑊𝐼𝑅2𝑁𝐼𝑅+𝑆𝑊𝐼𝑅2

train['mean_siwsi'] = ((train['mean_nir'] - train['mean_swir2']) / (train['mean_nir'] + train['mean_swir2']))

### Tasseled Cap Brightness

In [ ]:
train['tasseled_cap_brightness'] =  (0.3037 * train['mean_blue'] + 0.2793 * train['mean_green'] + 0.4743 * \
                                     train['mean_red'] + 0.5585 * train['mean_nir'] + 0.5082 * \
                                     train['mean_swir1'] + 0.1863 * train['mean_swir2'])



### Tasseled Cap Greenness

In [ ]:
train['tasseled_cap_greenness'] =  (-0.2848 * train['mean_blue'] - 0.2435 * train['mean_green'] - 0.5436 * train['mean_red'] + \
                                    0.7243 * train['mean_nir'] + 0.0840 * train['mean_swir1'] - 0.1800 * train['mean_swir2'])


### Tasseled Cap Wetness

In [ ]:
train['tasseled_cap_wetness'] = (0.1509 * train['mean_blue'] + 0.1973 * train['mean_green']  + 0.3279 * train['mean_red'] + \
                                0.3406 * train['mean_nir'] - 0.7112 * train['mean_swir1'] - 0.4572 * train['mean_swir2'])


## Climatic Factor

### Land Surface Temperature (LST)

##### Land Surface Temperature Function

In [ ]:
def calculate_land_surface_temperature(train, K1=774.8853, K2=1321.0789, wavelength=11.5, rho=0.98):
    # Assuming 'mean_thermal' column is present in the 'train' DataFrame

    radiance = (train['mean_thermal'].values * 0.0003342) + 0.1

    # TOA Brightness Temperature
    TOA_brightness_temp = K2 / np.log((K1 / radiance) + 1)

    # Land Surface Temperature
    LST = TOA_brightness_temp / (1 + (TOA_brightness_temp * wavelength) / (rho * 1.4388e4) * np.log((0.98 * 1.4388e4 * wavelength**5) / radiance + 1))

    train['land_surface_temp'] = LST

##### Applying LST Function

In [ ]:
# calculate LST on train dataset
calculate_land_surface_temperature(train)

### Bands Combination

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

# Load example data (you can replace this with your own dataset)
features = ['mean_blue', 'mean_green', 'mean_red',
            'mean_nir', 'mean_swir1', 'mean_swir2', 'mean_thermal', 'Code']


df = train[features]

markers =  {0: "s", 1: "X", 2: "P", 3: "o" }

# Create scatterplot matrix
sns.set(style="dark")
sns.pairplot(df, hue="Code", markers=markers)

# Show the plot
plt.show()

## Generating Indices using Interesting Bands

In [ ]:
interesting_bands = []

### Making Band combinations of the bands we find interesting
bands = ['mean_nir', 'mean_swir1', 'mean_swir2', 'mean_thermal']

# Get all possible two combinations
two_combinations = list(combinations(bands, 2))

# Print the result
for combo in two_combinations:
    interesting_bands.append(combo)
interesting_bands

In [ ]:
for combo in interesting_bands:
    train[combo[0]+'_'+combo[1]+ '_ratio'] = train[combo[0]] - (train[slope_col[0]] * train[combo[1]])
    train[combo[1]+'_'+combo[0]+ '_ratio'] = train[combo[1]] - (train[slope_col[0]] * train[combo[0]])


##  Training Models (RF, XGBoost, LMT, CatBoost, BRT and LightGBM) on Dataset


In [ ]:
training_columns = [col for col in train.columns
                    if col not in not_needed_columns and col not in bands_list]
training_columns.remove('Code')

print(len(training_columns))

In [ ]:
training_dataset = train[training_columns]


# Replace spaces with underscores
training_dataset.columns = training_dataset.columns.str.replace(' ', '_')

# Remove other special characters
training_dataset.columns = training_dataset.columns.str.replace('[^a-zA-Z0-9]', '', regex=True)


training_dataset = training_dataset.loc[:, ~training_dataset.columns.duplicated()]


In [ ]:
stratified_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# X, y  = training_dataset, train['Code']

X, y  =  training_dataset, train['Code']



feats = pd.DataFrame({'features': X.columns})
gbm_predictions = []


# F1 cv scores
xgb_cv_score_ = 0
lgb_cv_score_ = 0
cat_cv_score_ = 0
rf_cv_score_ = 0
lmt_cv_score_ = 0
brt_cv_score_ = 0

# precision cv score
xgb_precision_score_ = 0
lgb_precision_score_ = 0
cat_precision_score_ = 0
rf_precision_score_ = 0
lmt_precision_score_ = 0
brt_precision_score_ = 0

# recall cv score
xgb_recall_score_ = 0
lgb_recall_score_ = 0
cat_recall_score_ = 0
rf_recall_score_ = 0
lmt_recall_score_ = 0
brt_recall_score_ = 0


for i,(tr_index,test_index) in enumerate(stratified_kf.split(X,y)):
    print()
    print(f'######### FOLD {i+1} / {stratified_kf.n_splits} ')

    X_train,y_train = X.iloc[tr_index,:],y[tr_index]
    X_test,y_test = X.iloc[test_index,:],y[test_index]


    # Random forest model
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train,y_train)
    rf_cv_score_ += f1_score(y_test, rf.predict(X_test), average='weighted') / stratified_kf.n_splits
    rf_precision_score_ += precision_score(y_test, rf.predict(X_test), average='weighted') / stratified_kf.n_splits
    rf_recall_score_ += recall_score(y_test, rf.predict(X_test), average='weighted') / stratified_kf.n_splits



    # for xgb model
    xgb_gbm = xgb.XGBClassifier(missing=-1,device='cuda')
    xgb_gbm.fit(X_train,y_train, eval_set = [(X_test, y_test)], verbose=False)
    xgb_cv_score_ += f1_score(y_test, xgb_gbm.predict(X_test), average='weighted') / stratified_kf.n_splits
    xgb_precision_score_ += precision_score(y_test, xgb_gbm.predict(X_test), average='weighted') / stratified_kf.n_splits
    xgb_recall_score_ += recall_score(y_test, xgb_gbm.predict(X_test), average='weighted') / stratified_kf.n_splits


    #catboost model
    cat_gbm = CatBoostClassifier(random_seed=24,silent=True)
    cat_gbm.fit(X_train,y_train,eval_set = [(X_test, y_test)], verbose=False)
    cat_cv_score_ += f1_score(y_test, cat_gbm.predict(X_test), average='weighted') / stratified_kf.n_splits
    cat_precision_score_ += precision_score(y_test, cat_gbm.predict(X_test), average='weighted') / stratified_kf.n_splits
    cat_recall_score_ += recall_score(y_test, cat_gbm.predict(X_test), average='weighted') / stratified_kf.n_splits


    # Boosteed regression tree model
    brt_model = GradientBoostingClassifier(random_state=42)
    brt_model.fit(X_train,y_train)
    brt_cv_score_ += f1_score(y_test, brt_model.predict(X_test), average='weighted') / stratified_kf.n_splits
    brt_precision_score_ += precision_score(y_test, brt_model.predict(X_test), average='weighted') / stratified_kf.n_splits
    brt_recall_score_ += recall_score(y_test, brt_model.predict(X_test), average='weighted') / stratified_kf.n_splits


    # for lightgbm model
    warnings.filterwarnings("ignore", category=UserWarning, message="[LightGBM] \[Warning\] No further splits with positive gain, best gain: -inf")
    params  = {'random_state':42,'verbose': -1}
    lgb_gbm = lgb.LGBMClassifier(**params)
    lgb_gbm.fit(X_train,y_train,eval_set = [(X_test, y_test)])
    lgb_cv_score_ += f1_score(y_test, lgb_gbm.predict(X_test), average='weighted') / stratified_kf.n_splits
    lgb_precision_score_ += precision_score(y_test, lgb_gbm.predict(X_test), average='weighted') / stratified_kf.n_splits
    lgb_recall_score_ += recall_score(y_test, lgb_gbm.predict(X_test), average='weighted') / stratified_kf.n_splits


print( ' RF CV F1 Score : ', rf_cv_score_)
print( ' XGB CV F1 Score : ', xgb_cv_score_)
print( ' LGB CV F1 Score : ', lgb_cv_score_)
print( ' CAT CV F1 Score  : ', cat_cv_score_)
print( ' BRT CV F1 Score  : ', brt_cv_score_)

In [ ]:
f1_scores = [rf_cv_score_, xgb_cv_score_, lgb_cv_score_, brt_cv_score_, cat_cv_score_]
precision_scores = [rf_precision_score_, xgb_precision_score_, lgb_precision_score_, brt_precision_score_, cat_precision_score_]
recall_scores = [rf_recall_score_, xgb_recall_score_, lgb_recall_score_, brt_recall_score_, cat_recall_score_]

model_names = ['Random Forest', 'XGBoost', 'LightGBM', 'BRT', 'CatBoost']

evaluation_df = pd.DataFrame({'Model': model_names,  'Precision Score': precision_scores, 'Recall Score': recall_scores, 'F1 Score': f1_scores})

evaluation_df.sort_values(by='F1 Score', ascending=False)

## Columns used for training as determined by  Recursive Feature Elimination Algorithm

In [ ]:
training_columns = ['Year',
 'WDVI',
 'nightlights.average_min',
 'mean_nir_mean_swir2_ratio',
 'nightlights.average_max',
 'Coor_y',
 'clm_std',
 'mean_nir_mean_thermal_ratio',
 'mean_swir1_mean_thermal_ratio',
 'nightlights.average_viirs.v21_m_500m_s_{year}0101_{year}1231_go_epsg4326_v20230318',
 'nightlights.average_mean',
 'mean_thermal_mean_swir2_ratio',
 'fapar_max',
 'bs_mean',
 'mean_swir1_mean_swir2_ratio',
 'ndsi_min',
 'bioclim.var_chelsa.bio2_m_1km_s_1981_2010_go_epsg.4326_v20231002',
 'clm_max',
 'evi_max',
 'dtm_max',
 'bioclim.var_min',
 'clm_lst_mod11a2.daytime_p95_1km_s0..0cm_{year}_v1.2',
 'clm_min',
 'bs_max',
 'bs_min',
 'dtm_skew',
 'bs_glad.landsat.seasconv.m.yearly_sum_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20231127',
 'mean_swir2_mean_thermal_ratio',
 'ndvi_glad.landsat.seasconv.m.yearly_p75_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20231127',
 'pos.openess_min',
 'evi_glad.landsat.seasconv_m_30m_s_{year}0501_{year}0630_eu_epsg.3035_v20231127',
 'wv_mcd19a2v061.seasconv.m.yearly_p25_1km_s_{year}0101_{year}1231_go_epsg.4326_v20230619',
 'mean_thermal_mean_swir1_ratio',
 'Coor_x',
 'pos.openess_max',
 'dtm_elev.lowestmode_gedi.eml_mf_30m_0..0cm_2000..2018_eumap_epsg3035_v0.3',
 'pos.openess_merit.dem_m_250m_s0..0cm_2017_2017_go_epsg.4326_v20231002',
 'mean_nir_mean_swir1_ratio',
 'ndsi_glad.landsat.seasconv_m_30m_s_{year}0701_{year}0831_eu_epsg.3035_v20231127',
 'pos.openess_mean',
 'dtm_kurt',
 'mean_swir2_mean_swir1_ratio',
 'wv_min',
 'dtm_mean',
 'bioclim.var_chelsa.bio3_m_1km_s_1981_2010_go_epsg.4326_v20231002',
 'tasseled_cap_brightness',
 'evi_std',
 'wv_mean',
 'clm_lst_mod11a2.nighttime_sd_1km_s0..0cm_{year}_v1.2',
 'landform_mean',
 'ndvi_max',
 'clm_lst_mod11a2.nighttime_p95_1km_s0..0cm_{year}_v1.2',
 'ndsi_glad.landsat.seasconv_m_30m_s_{year}0501_{year}0630_eu_epsg.3035_v20231127',
 'clm_lst_mod11a2.nighttime_p05_1km_s0..0cm_{year}_v1.2',
 'wv_max',
 'landform.dissected.terrace.moderate.plateau_mean',
 'wv_skew',
 'wv_mcd19a2v061.seasconv.m.yearly_sd_1km_s_{year}0101_{year}1231_go_epsg.4326_v20230619',
 'slope_nunique',
 'clm_skew',
 'vbf_merit.dem_m_250m_s0..0cm_2017_2017_go_epsg.4326_v20231002',
 'dtm_std',
 'landform_terrain.class_c_250m_s0..0cm_2017_2018_go_epsg.4326_v20231002',
 'ndsi_glad.landsat.seasconv_m_30m_s_{year}0901_{year}1031_eu_epsg.3035_v20231127',
 'slope_std',
 'fapar_glad.landsat.seasconv_m_30m_s_{year}0301_{year}0430_eu_epsg.3035_v20231127',
 'snow.prob_mean',
 'mean_swir1',
 'fapar_std',
 'twi',
 'clm_lst_mod11a2.daytime_p05_1km_s0..0cm_{year}_v1.2',
 'evi_glad.landsat.seasconv_m_30m_s_{year}0701_{year}0831_eu_epsg.3035_v20231127',
 'bioclim.var_kurt',
 'bioclim.var_max',
 'vbf_mean',
 'lcv_mean',
 'landform_skew',
 'ndvi_glad.landsat.seasconv.m.yearly_p50_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20231127',
 'fapar_glad.landsat.seasconv_m_30m_s_{year}0501_{year}0630_eu_epsg.3035_v20231127',
 'fapar_glad.landsat.seasconv_m_30m_s_{year}0901_{year}1031_eu_epsg.3035_v20231127',
 'mean_savi',
 'clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_{year}_v1.2',
 'lcv_globalcropland_bowen.et.al_p_1km_s0..0cm_{year}_v0.1',
 'wetlands.regularly-flooded_upmc.wtd_p_250m_b0..200cm_2010_2015_go_epsg.4326_v20231002',
 'mean_thermal',
 'snow.prob_esacci.m12_p90_500m_s_2000_2012_go_epsg.4326_v20231002',
 'snow.prob_std',
 'evi_glad.landsat.seasconv_m_30m_s_{year}0301_{year}0430_eu_epsg.3035_v20231127',
 'wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s_{year}0101_{year}1231_go_epsg.4326_v20230619',
 'landform.middle.large.slope_terrain.class_p_250m_s0..0cm_2017_2018_go_epsg.4326_v20231002',
 'mean_swir2_mean_nir_ratio',
 'vbf_max',
 'ndvi_nunique',
 'mean_nir',
 'clm_accum.precipitation_chelsa.annual_m_1km_s0..0cm_{year}_v2.1',
 'clm_lst_mod11a2.daytime_p50_1km_s0..0cm_{year}_v1.2',
 'snow.prob_max',
 'snow.prob_esacci.m10_p90_500m_s_2000_2012_go_epsg.4326_v20231002',
 'slope_merit.dem_m_250m_s0..0cm_2017_2017_go_epsg.4326_v20231002',
 'bioclim.var_skew',
 'fapar_glad.landsat.seasconv_m_30m_s_{year}0701_{year}0831_eu_epsg.3035_v20231127',
 'snow.prob_esacci.m01_p90_500m_s_2000_2012_go_epsg.4326_v20231002',
 'snow.prob_esacci.m10_sd_500m_s_2000_2012_go_epsg.4326_v20231002',
 'bioclim.var_chelsa.bio12_m_1km_s_1981_2010_go_epsg.4326_v20231002',
 'mean_blue',
 'snow.prob_min',
 'wv_std',
 'fapar_mean',
 'vbf_min',
 'snow.prob_esacci.m09_sd_500m_s_2000_2012_go_epsg.4326_v20231002',
 'slope_min',
 'lcv_std',
 'snow.prob_esacci.m12_sd_500m_s_2000_2012_go_epsg.4326_v20231002',
 'lcv_max',
 'dtm_lithology_usgs.ecotapestry.intermediate.plutonics_p_250m_s0..0cm_2014_v1.0',
 'clm_nunique',
 'clm_lst_mod11a2.daytime_sd_1km_s0..0cm_{year}_v1.2',
 'dtm_lithology_usgs.ecotapestry.siliciclastic.sedimentary_p_250m_s0..0cm_2014_v1.0',
 'ndsi_glad.landsat.seasconv_m_30m_s_{year}0101_{year}0228_eu_epsg.3035_v20231127',
 'upslope.curvature_min',
 'ndsi_glad.landsat.seasconv_m_30m_s_{year}0301_{year}0430_eu_epsg.3035_v20231127',
 'wetlands.regularly-flooded_min',
 'ndsi_mean',
 'wv_kurt',
 'downslope.curvature_mean',
 'clm_mean',
 'wetlands.cw_upmc.wtd_c_250m_b0..200cm_2010_2015_go_epsg.4326_v20231002',
 'downslope.curvature_merit.dem_m_250m_s0..0cm_2017_2017_go_epsg.4326_v20231002',
 'ndvi_glad.landsat.seasconv.m.yearly_p25_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20231127',
 'log1p.upstream.area_merit.hydro_m_250m_b0..0cm_2017_2017_go_epsg.4326_v20231002',
 'snow.prob_esacci.m03_p90_500m_s_2000_2012_go_epsg.4326_v20231002',
 'log1p.upstream.area_min',
 'land_surface_temp',
 'clm_kurt',
 'snow.prob_esacci.m11_sd_500m_s_2000_2012_go_epsg.4326_v20231002',
 'ndsi_std',
 'ndvi_mean',
 'upslope.curvature_max',
 'snow.prob_esacci.m03_sd_500m_s_2000_2012_go_epsg.4326_v20231002',
 'mean_red',
 'wetlands.regularly-flooded_mean',
 'slope_mean',
 'ls_factor',
 'snow.prob_esacci.m05_sd_500m_s_2000_2012_go_epsg.4326_v20231002',
 'mean_swir1_mean_nir_ratio',
 'fapar_glad.landsat.seasconv_m_30m_s_{year}0101_{year}0228_eu_epsg.3035_v20231127',
 'wv_mcd19a2v061.seasconv.m.yearly_p50_1km_s_{year}0101_{year}1231_go_epsg.4326_v20230619',
 'downslope.curvature_max',
 'snow.prob_esacci.m01_sd_500m_s_2000_2012_go_epsg.4326_v20231002',
 'wetlands.regularly-flooded_max',
 'fapar_min',
 'log1p.upstream.area_mean',
 'dtm_lithology_usgs.ecotapestry.acid.volcanic_p_250m_s0..0cm_2014_v1.0',
 'snow.prob_esacci.m11_p90_500m_s_2000_2012_go_epsg.4326_v20231002',
 'snow.prob_esacci.m02_p90_500m_s_2000_2012_go_epsg.4326_v20231002']


print(len(training_columns))

### Clean the names of the columns

In [ ]:
training_dataset = train[training_columns]

# Replace spaces with underscores
training_dataset.columns = training_dataset.columns.str.replace(' ', '_')

# Remove other special characters
training_dataset.columns = training_dataset.columns.str.replace('[^a-zA-Z0-9]', '', regex=True)


training_dataset = training_dataset.loc[:, ~training_dataset.columns.duplicated()]


# Modelling

### Voting Classifier Cross Validation

In [ ]:
stratified_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X, y  = training_dataset, train['Code']

feats = pd.DataFrame({'features': X.columns})
gbm_predictions = []
cv_score_ = 0
oof_preds = np.zeros((train.shape[0],))


all_preds = []
all_true_labels = []

for i,(tr_index,test_index) in enumerate(stratified_kf.split(X,y)):
    print()
    print(f'######### FOLD {i+1} / {stratified_kf.n_splits} ')

    X_train,y_train = X.iloc[tr_index,:],y[tr_index]
    X_test,y_test = X.iloc[test_index,:],y[test_index]


    # for xgb model
    xgb_gbm = xgb.XGBClassifier(
                            gamma = 0.23,
                            n_estimators=2000,
                            max_depth=12,
                            learning_rate=0.01,
                            subsample=0.8,
                            colsample_bytree=0.4,
                            missing=-1,
                            device='cuda',
                            )


    # for lightgbm model
    warnings.filterwarnings("ignore", category=UserWarning, message="[LightGBM] \[Warning\] No further splits with positive gain, best gain: -inf")
    params  = {
   'boosting_type': 'dart','n_estimators':2500,'objective': 'multiclass',
    'learning_rate':0.02, 'num_leaves':15,'reg_alpha':0,'reg_lambda':7,
    'max_depth':5,
    'random_state':42,'verbose': -1}
    lgb_gbm = lgb.LGBMClassifier(**params)



    #catboost classifier
    cat_gbm = CatBoostClassifier(
        loss_function='MultiClass',
        depth = 6,
        task_type="GPU",
        learning_rate=0.01,
        iterations=4000,
        od_type="Iter",
        random_seed=24,
        silent=True,
    )


    # voting
    voting_classifier = VotingClassifier(
    estimators=[
        ('xgboost', xgb_gbm),
        ('lightgbm', lgb_gbm),
        ('catboost', cat_gbm),
    ],
    voting='soft',
    )
    voting_classifier.fit(X_train, y_train)

    cv_score_ += f1_score(y_test, voting_classifier.predict(X_test), average='weighted') / stratified_kf.n_splits
    fold_preds = voting_classifier.predict(X_test)
    all_preds.extend(fold_preds)
    all_true_labels.extend(y_test)


print(' CV F1_Score : ', cv_score_)


In [ ]:
overall_cm = confusion_matrix(all_true_labels, all_preds)
labels = ['No Gully/badland', 'Gully', 'Badland', 'Landslides']
plt.figure(figsize=(15, 5))
sns.heatmap(overall_cm, annot=True, fmt="d", cmap="crest", xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')


In [ ]:
string_class_preds = ['No Gully/badland' if label == 0 else
                'Gully' if label == 1 else 'Badland' if label == 2 else
                'Landslides' for label in all_preds]

string_class_true = ['No Gully/badland' if label == 0 else
                'Gully' if label == 1 else 'Badland' if label == 2 else
                'Landslides' for label in all_true_labels]

In [ ]:
report = classification_report(string_class_true, string_class_preds)
print(report)